In [21]:
import pandas as pd
import numpy as np
import pickle
import datetime
from sklearn.model_selection import KFold, train_test_split
import os

path="../data/"

os.getcwd()

'/home/cseadmin/undergrad1/dz/GISCUP2021/model'

In [22]:
train = pd.read_csv(path+'train/train.csv')
test = pd.read_csv(path+'test/test.csv')

test['date'] = '20200901'
train['date'] = pd.to_datetime(train['date'].astype('str'))
test['date'] = pd.to_datetime(test['date'])

for df in [train, test]:
    df['day_bias'] = (df['date']-pd.Timestamp(2020,8,1)).dt.days
    df['weekday'] = df['date'].dt.weekday<5
    df['cross_no'] = df['cross_no']*(df['cross_sum']>0)
    df['link_length'] = df['distance']/df['link_no']
    df['log_distance'] = np.log(df['distance'])
    df['log_simple_eta'] = np.log(df['simple_eta'])
    
    for i in range(5):
        df['current_no_ratio_'+str(i)] = df['current_no_'+str(i)]/df['link_no']
        df['current_time_ratio_'+str(i)] = df['current_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
        df['next_no_ratio_'+str(i)] = df['next_no_'+str(i)]/df['link_no']
        df['next_time_ratio_'+str(i)] = df['next_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
    for i in range(4):
        df['type_no_ratio_'+str(i)] = df['type_no_'+str(i)]/df['link_no']
        df['type_time_ratio_'+str(i)] = df['type_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])

In [23]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
print(train.info())
print(test.info())

Memory usage after optimization is: 2565.83 MB
Decreased by 74.1%
Memory usage after optimization is: 84.07 MB
Decreased by 74.5%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8651005 entries, 0 to 8651004
Columns: 151 entries, slice_id to type_time_ratio_3
dtypes: bool(1), datetime64[ns](1), float16(119), float32(1), int16(19), int32(4), int8(6)
memory usage: 2.5 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288076 entries, 0 to 288075
Columns: 151 entries, slice_id to type_time_ratio_3
dtypes: bool(1), datetime64[ns](1), float16(119), float32(1), int16(17), int32(3), int8(9)
memory usage: 84.1 MB
None


In [24]:
train.to_pickle(path+'train/train.pkl')
test.to_pickle(path+'test/test.pkl')